<a href="https://colab.research.google.com/github/ywanglab/STAT4160/blob/main/notebooks/lec2_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab cell
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Adjust these two for YOUR repo
REPO_OWNER = "ywanglab"
REPO_NAME  = "STAT4160"   # e.g., unified-stocks-team1

BASE_DIR   = "/content/drive/MyDrive/dspt25"
CLONE_DIR  = f"{BASE_DIR}/{REPO_NAME}"
REPO_URL   = f"https://github.com/{REPO_OWNER}/{REPO_NAME}.git"

import os, pathlib
pathlib.Path(BASE_DIR).mkdir(parents=True, exist_ok=True)


In [ ]:
import os, subprocess, shutil, pathlib

if not pathlib.Path(CLONE_DIR).exists():
    !git clone {REPO_URL} {CLONE_DIR}
else:
    # If the folder exists, just ensure it's a git repo and pull latest
    os.chdir(CLONE_DIR)
    !git status
    !git pull --ff-only
os.chdir(CLONE_DIR)
print("Working dir:", os.getcwd())

Refresh index: 100% (32/32), done.
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Already up to date.
Working dir: /content/drive/MyDrive/dspt25/STAT4160


In [ ]:
# Replace with your name and school email
!git config user.name "Yi Wang"
!git config user.email "ywang2@aum.edu"

!git config --get user.name
!git config --get user.email

Yi Wang
ywang2@aum.edu


In [ ]:
BRANCH = "setup/git-lfs"
!git checkout -b {BRANCH}
!git branch --show-current

Switched to a new branch 'setup/git-lfs'
setup/git-lfs


In [ ]:
gitignore = """\
book/
.quarto/
/.quarto/

# Byte-compiled / cache
__pycache__/
*.py[cod]

# Jupyter checkpoints
.ipynb_checkpoints/

# OS/editor files
.DS_Store
Thumbs.db
.vscode/

# Environments & secrets
.env
.env.*
.venv/
*.pem
*.key

# Data (raw & interim never committed)
data/raw/
data/interim/

# Logs & caches
logs/
.cache/
"""
open(".gitignore", "w").write(gitignore)
print(open(".gitignore").read())

book/
.quarto/
/.quarto/

# Byte-compiled / cache
__pycache__/
*.py[cod]

# Jupyter checkpoints
.ipynb_checkpoints/

# OS/editor files
.DS_Store
Thumbs.db
.vscode/

# Environments & secrets
.env
.env.*
.venv/
*.pem
*.key

# Data (raw & interim never committed)
data/raw/
data/interim/

# Logs & caches
logs/
.cache/



In [ ]:
# Install git-lfs on the Colab VM (one-time per runtime)
!apt-get -y update >/dev/null. # refresh vailable packages from the repositories
!apt-get -y install git-lfs >/dev/null
!git lfs install
!git lfs version

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Updated git hooks.
Git LFS initialized.
git-lfs/3.0.2 (GitHub; linux amd64; go 1.18.1)


In [ ]:
# Add .gitattributes entries via git lfs track
!git lfs track "data/processed/*.parquet"
!git lfs track "data/*.db"
!git lfs track "models/*.pt"
!git lfs track "reports/*.pdf"

# Show what LFS is tracking and verify .gitattributes created
!git lfs track
print("\n.gitattributes:")
print(open(".gitattributes").read())

Tracking "data/processed/*.parquet"
Tracking "data/*.db"
Tracking "models/*.pt"
Tracking "reports/*.pdf"
Listing tracked patterns
    data/processed/*.parquet (.gitattributes)
    data/*.db (.gitattributes)
    models/*.pt (.gitattributes)
    reports/*.pdf (.gitattributes)
Listing excluded patterns

.gitattributes:
data/processed/*.parquet filter=lfs diff=lfs merge=lfs -text
data/*.db filter=lfs diff=lfs merge=lfs -text
models/*.pt filter=lfs diff=lfs merge=lfs -text
reports/*.pdf filter=lfs diff=lfs merge=lfs -text



In [ ]:
import pandas as pd, numpy as np, os, pathlib

pathlib.Path("data/processed").mkdir(parents=True, exist_ok=True)

tickers = pd.read_csv("tickers_25.csv")["ticker"].tolist() if os.path.exists("tickers_25.csv") else [
    "AAPL","MSFT","AMZN","GOOGL","META","NVDA","TSLA","JPM","JNJ","V",
    "PG","HD","BAC","XOM","CVX","PFE","KO","DIS","NFLX","INTC","CSCO","ORCL","T","VZ","WMT"
]

# 1000 business days x up to 25 tickers ~ 25k rows; a few MB as Parquet
dates = pd.bdate_range("2018-01-01", periods=1000)
df = (pd.MultiIndex.from_product([tickers, dates], names=["ticker","date"])
      .to_frame(index=False))
rng = np.random.default_rng(42)
df["r_1d"] = rng.normal(0, 0.01, size=len(df))  # synthetic daily returns
df.to_parquet("data/processed/sample_returns.parquet", index=False)
df.head()

,ticker,date,r_1d
0,AAPL,2018-01-01,0.003047
1,AAPL,2018-01-02,-0.010400
2,AAPL,2018-01-03,0.007505
3,AAPL,2018-01-04,0.009406
4,AAPL,2018-01-05,-0.019510


In [ ]:
!git add .gitignore .gitattributes data/processed/sample_returns.parquet
!git status

!git commit -m "feat: add .gitignore and git-lfs tracking; add sample Parquet"
!git log --oneline -n 2

# if see the error of cannot run .git/hooks/post-commit: chmod +x .git/hooks/post-commit

On branch setup/git-lfs
nothing to commit, working tree clean
On branch setup/git-lfs
nothing to commit, working tree clean
f839d76 (HEAD -> setup/git-lfs) feat: add .gitignore and git-lfs tracking; add sample Parquet
3c826a1 (origin/main, origin/HEAD, main) added lec2.html


In [ ]:
# Colab cell: push using a temporary URL with token (not saved to git config)
from getpass import getpass
token = getpass("Enter your GitHub token (input hidden; not stored): ")

push_url = f"https://{token}@github.com/{REPO_OWNER}/{REPO_NAME}.git"
!git push {push_url} {BRANCH}:{BRANCH} # local branch: remote branch

# Optional: immediately clear the token variable
del token
# if error:cannot exec '.git/hooks/pre-push': chmod +x .git/hooks/pre-push


Enter your GitHub token (input hidden; not stored): ··········
Uploading LFS objects: 100% (1/1), 260 KB | 0 B/s, done.
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 935 bytes | 44.00 KiB/s, done.
Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: 
remote: Create a pull request for 'setup/git-lfs' on GitHub by visiting:
remote:      https://github.com/ywanglab/STAT4160/pull/new/setup/git-lfs
remote: 
To https://github.com/ywanglab/STAT4160.git
 * [new branch]      setup/git-lfs -> setup/git-lfs


In [ ]:
# OPTIONAL: Create PR via GitHub API (requires token again)
from getpass import getpass
import requests, json

token = getpass("GitHub token (again, not stored): ")
headers = {"Authorization": f"Bearer {token}",
           "Accept": "application/vnd.github+json"}
payload = {
    "title": "Setup: .gitignore + Git-LFS + sample Parquet",
    "head": BRANCH,
    "base": "main",
    "body": "Adds .gitignore, configures Git-LFS for parquet/db/pdf/model files, and commits a sample Parquet for verification."
}
r = requests.post(f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/pulls",
                  headers=headers, data=json.dumps(payload))
print("PR status:", r.status_code)
try:
    pr_url = r.json()["html_url"]
    print("PR URL:", pr_url)
except Exception as e:
    print("Response:", r.text)
del token

GitHub token (again, not stored): ··········
PR status: 422
Response: {"message":"Validation Failed","errors":[{"resource":"PullRequest","field":"head","code":"invalid"}],"documentation_url":"https://docs.github.com/rest/pulls/pulls#create-a-pull-request","status":"422"}


In [ ]:
!git lfs ls-files

e87eb77993 * data/processed/sample_returns.parquet


In [ ]:
# List LFS-tracked files (shows their OIDs and sizes)
!git lfs ls-files -l

e87eb77993ba9e8a89e206b2623747f9c86b18fd9ccb6d12735f43da582b0720 * data/processed/sample_returns.parquet


In [ ]:
# Show the pointer text that’s in Git history
!git cat-file -p HEAD:data/processed/sample_returns.parquet

version https://git-lfs.github.com/spec/v1
oid sha256:e87eb77993ba9e8a89e206b2623747f9c86b18fd9ccb6d12735f43da582b0720
size 259949


In [ ]:
# Ensure you have the actual binary downloaded
# !git lfs fetch --include="data/processed/*.parquet"
# !git lfs checkout   # replaces pointers in your working tree with real files
